In [1]:
!pip install adversarial-robustness-toolbox > /dev/null

In [62]:
import tarfile
from urllib.request import urlopen
from os import makedirs, listdir
from os.path import join, exists
import json
import time

import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
import csv
# from tqdm import tqdm
from tqdm.notebook import tqdm

from tensorflow.keras.layers import Input
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.applications import inception_v3
from art.estimators.classification import TensorFlowV2Classifier

from art.attacks.evasion import FastGradientMethod
from art.attacks.evasion import ProjectedGradientDescent
from art.attacks.evasion import BasicIterativeMethod
from art.attacks.evasion import CarliniL2Method
from art.attacks.evasion import CarliniLInfMethod

In [3]:
dataset_label = "Imagenette"
input_dataset_link = "https://s3.amazonaws.com/fast-ai-imageclas/imagenette2.tgz"
labels_link = "https://storage.googleapis.com/download.tensorflow.org/data/imagenet_class_index.json"
imagenette_directory = "./imagenette2/train"

input_dataset_content = urlopen(input_dataset_link)
input_dataset_tar = tarfile.open(fileobj=input_dataset_content, mode="r|gz")
input_dataset_tar.extractall()

labels_content = urlopen(labels_link)
labels = json.loads(labels_content.read()) 

labels = {int(index): value for index,value in labels.items()}
class_to_index = {value[0]: int(index) for index,value in labels.items()}

In [4]:
org_dataset_directory = f"./{dataset_label}/Original"
adv_dataset_directory = f"./{dataset_label}/Adversarial"
metadata_file = f"./{dataset_label}/metadata.csv"
metadata_fields = [
  "org_image", "adv_image", "adv_npz", 
  "attack_type", "eps", "eps_step", "max_iter", "attack_time",
  'org_label', 'adv_label',
  "status"
]
for top_k in range(5):
    metadata_fields.append(f'org_top{top_k+1}_index')
    metadata_fields.append(f'org_top{top_k+1}_prob')
    metadata_fields.append(f'adv_top{top_k+1}_index')
    metadata_fields.append(f'adv_top{top_k+1}_prob')

In [6]:
image_size = (300, 300, 3)
pixel_range = 2 # range of processed image will be -1 to 1

eps(02)_eps_step(01)


In [7]:
inception_model = inception_v3.InceptionV3(input_tensor=Input(shape=image_size),
                                 include_top=True, 
                                 weights='imagenet', 
                                 classifier_activation='softmax')

loss = CategoricalCrossentropy(from_logits=False)

classifier = TensorFlowV2Classifier(model=inception_model,
                                    nb_classes=1000,
                                    loss_object=loss,
                                    input_shape=image_size,
                                    clip_values=(-1,1))

2022-12-31 07:07:38.235127: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-31 07:07:38.236687: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-31 07:07:38.237671: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-31 07:07:38.238798: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

96124928/96112376 [==============================] - 1s 0us/step


In [9]:
adv_dataset_images = join(adv_dataset_directory,attack_type,'Images')
adv_dataset_npz = join(adv_dataset_directory,attack_type,'NPZ')
if not exists(adv_dataset_images): makedirs(adv_dataset_images)
if not exists(adv_dataset_npz): makedirs(adv_dataset_npz)

if not exists(metadata_file):
    with open(metadata_file, 'w') as metadata_csv:
        metadata_writer = csv.writer(metadata_csv)
        metadata_writer.writerow(metadata_fields)

In [10]:
def preprocess_input(image):
    return inception_v3.preprocess_input(image)

def inverse_preprocess(image):
    image = np.copy(image)
    image += 1.0
    image *= 127.5
    image = image.astype(np.uint8)
    return image

def predict(X, name=False):
    if len(X.shape) == 3:
        prediction = np.argmax(classifier.predict(
                        np.expand_dims(X, axis=0)
                    ), axis=1)

        if name: return labels[prediction[0]][0]
        else: return prediction[0]
    else:
        predictions = np.argmax(classifier.predict(X), axis=1)
        if name: return map(lambda prediction: labels[prediction][0], predictions)
        else: return predictions

def predict_top5(image):
    y_pred = classifier.predict(np.expand_dims(image, axis=0))[0]
    top5_inds = np.argsort(y_pred)[-5:][::-1]
    top5_probs = y_pred[top5_inds] 
    return [{'index':ind, 'prob':prob} for ind,prob in zip(top5_inds, top5_probs)]
    
def is_generated(adv_image_file):
    metadata = pd.read_csv(metadata_file)
    rows = metadata[metadata['adv_image'] == adv_image_file]
    return len(rows) > 0

def write_metadata(row_dict):
    with open(metadata_file, "a") as metadata_csv:
        metadata_writer = csv.DictWriter(metadata_csv, fieldnames=metadata_fields)
        metadata_writer.writerow(row_dict)
        metadata_csv.close()

In [30]:
attack_params = {
    'eps': (1/100) * pixel_range,
    'eps_step': 0.01,
    'max_iter': 10
}
attack_params_str = '_'.join([f"{param_name}({str(param_val).split('.')[-1]})" for param_name, param_val in attack_params.items()])

attack = FastGradientMethod(estimator=classifier, **attack_params)
# attack = ProjectedGradientDescent(estimator=classifier, **attack_params, verbose=False)
# attack = CarliniL2Method(classifier=classifier,  **attack_params, verbose=False)
# attack = CarliniLInfMethod(classifier=classifier, **attack_params, verbose=False)
attack_type = attack.__class__.__name__

In [68]:
# For debug purpose
reconstructable = 0
attack_done = 0

# Generating Original Dataset
classes_dir = sorted(listdir(imagenette_directory))
for c_index, class_name in enumerate(tqdm(classes_dir, desc='Class', position=0)):

    image_files = sorted(listdir(join(imagenette_directory, class_name)))
    inner_tqdm = tqdm(image_files, position=1, leave=False)
    for i_index, image_file in enumerate(inner_tqdm):        
        inner_tqdm.set_description(f'Attacks: {attack_done} Reconstructable: {reconstructable}')
        
        # Creating all filenames
        image_file_without_ext = image_file.split(".")[0]
        org_image_file = join(imagenette_directory, class_name, image_file)
        adv_image_file = join(adv_dataset_images, f'adv_{attack_type}_{attack_params_str}_{image_file_without_ext}.png')
        adv_npz_file = join(adv_dataset_npz, f'adv_{attack_type}_{attack_params_str}_{image_file_without_ext}.npz')
        
        # Initializing Metadata 
        metadata = {}
        metadata['org_image'] = org_image_file
        metadata['org_label'] = class_name
    
        image = Image.open(org_image_file)
    
        # Filtering out images which are grayscaled
        if image.getbands() == ('L',):
            metadata['status'] = 'GRAYSCALED'
            write_metadata(metadata)
            continue
    
        # Rescaling images to make all image sizes same
        org_image = np.asarray(image.resize((image_size[0], image_size[1])))
        org_image_processed = preprocess_input(org_image)
        
        # Saving top 5 values to metadata
        org_top5 = predict_top5(org_image_processed)
        for topk, topk_item in enumerate(org_top5):
            metadata[f'org_top{topk+1}_index'] = topk_item['index']
            metadata[f'org_top{topk+1}_prob'] = topk_item['prob']
        
        # Filtering out images which are not correctly classified
        y_pred = predict(org_image_processed, name=True)
        if y_pred != class_name: 
            metadata['status'] = 'NOT_CORRECTLY_CLASSIFIED'
            write_metadata(metadata)
            continue
        
        # Saving attack params to metadata
        metadata['attack_type'] = attack_type
        if 'eps' in attack.attack_params: metadata['eps'] = attack_params['eps']
        if 'eps_step' in attack.attack_params: metadata['eps_step'] = attack_params['eps_step']
        if 'max_iter' in attack.attack_params: metadata['max_iter'] = attack_params['max_iter']
        
        # Generating adversarial version
        start = time.perf_counter()
        adv_image_processed = attack.generate(x=np.expand_dims(org_image_processed, axis=0))[0]
        end = time.perf_counter()

        # Filtering out images which are not successfully attacked
        y_adv = predict(adv_image_processed, name=True)
        if y_adv == class_name:
            metadata['status'] = 'ATTACK_FAILED'
            write_metadata(metadata)
            continue
    
        # Saving NPZ file
        attack_done += 1
        np.savez_compressed(adv_npz_file, image=adv_image_processed)
        metadata['adv_npz'] = adv_npz_file
            
        # Saving Attack result to metadata
        metadata['adv_label'] = y_adv
        metadata['attack_time'] = round(end-start, 2)
        adv_top5 = predict_top5(adv_image_processed)
        for topk, topk_item in enumerate(adv_top5):
            metadata[f'adv_top{topk+1}_index'] = topk_item['index']
            metadata[f'adv_top{topk+1}_prob'] = topk_item['prob']
        
        # Checking if the attack image generated can be converted to pixelized image
        adv_image = inverse_preprocess(adv_image_processed)
        adv_image_reconstructed = preprocess_input(adv_image)
        
        # Filtering out non-reconstrctable images
        y_adv_reconstructed = predict(adv_image_reconstructed, name=True)
        if y_adv != y_adv_reconstructed: 
            metadata['status'] = 'CAN_NOT_BE_RECONSTRUCTED'
            write_metadata(metadata)
            continue

        # Saving image
        reconstructable += 1
        Image.fromarray(adv_image).save(adv_image_file)
        metadata['adv_image'] = adv_image_file

        # Writing metadata to file
        metadata['status'] = 'SUCCESSFULL'
        write_metadata(metadata)

Class:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/963 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [51]:
metadata =pd.read_csv(metadata_file)
# metadata[metadata.adv_image.notna()][["org_image","adv_image"]]
# list(metadata[metadata.org_image == metadata.iloc[2]['org_image']]['adv_image'])